# Gym create Environment

### Imports

In [1]:
import gym 
from gym import Env
from gym.spaces import Discrete, Box, Dict, Tuple, MultiBinary, MultiDiscrete

import random
import pandas as pd
import numpy as np
import random
import os

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.evaluation import evaluate_policy

### Hulponderdelen environment
In het environment van een 3 bij 3 wordt gewerkt met een matrix en een lisjt met containernummers die ingedeeld moeten worden.

In [2]:
def create_matrix(x):
    data = {}
    for i in range(0,x):
        data[i] = [0]*x
    nullen_matrix = pd.DataFrame(data)
    return nullen_matrix

In [3]:
create_matrix(3)

,0,1,2
0,0,0,0
1,0,0,0
2,0,0,0


In [4]:
list_containers = list([1,1,1,2,2,2,3,3,3])
random.shuffle(list_containers)

In [5]:
print(list_containers)

[3, 1, 3, 1, 1, 2, 2, 2, 3]


### Environment maken
In dit model wordt gewerkt met een hele simpele reward: er wordt gekeken of een container niet op een plek wordt geplaatst waar er al 1 staat. Alleen deze restricite wordt op getraind, nog neit een optimale oplossing, dus welke containers waar staan maakt nu nog niet uit.

In [6]:
class Mijn_Env(Env):
    def __init__(self):
        self.x = 3
        self.midden_env = 1
        
        self.action_space = MultiDiscrete([3,3])
        self.observation_space = Box(low=int(0),high=int(9),shape=(3,3), dtype=int)
        #state: 3 bij 3 matrix in df vorm
        self.state = create_matrix(self.x) #achteraf naar numpy
        
        self.list_containers = list([1,1,1,2,2,2,3,3,3])
        random.shuffle(self.list_containers)
        
        self.duration = 9 # want 9 containers plaatsen
        
    def step(self, action):
        self.duration -= 1
        
        # container pakken:
        container = self.list_containers.pop()
        # bij at is het [y,x]. action is een [i,j] coördinaat (Multidiscrete)
        y_row = action[0]
        x_col = action[1]
        plek = self.state.iloc[y_row,x_col]
        
        # REWARD
        reward = 0
        #leeg vakje?
        if plek == 0:
            reward += 1
        else:
            reward -= 1
        
        # PLAATSEN
        self.state.iloc[y_row, x_col] = container
        
        # STOPPEN
        if self.duration <= 0:
            done=True
        else:
            done=False
            
        info = {}
        
        return self.state.to_numpy(), reward, done, info
    
    def render(self, mode = 'human'):
        display(self.state)
    
    def reset(self):
        # terug naar lege matrix:
        self.state = create_matrix(self.x)
        # nieuwe random list_containers
        self.list_containers = list([1,1,1,2,2,2,3,3,3])
        random.shuffle(self.list_containers)
        # duration terug op 9
        self.duration = 9
        
        return self.state.to_numpy()

In [7]:
env = Mijn_Env()

In [8]:
env.state

,0,1,2
0,0,0,0
1,0,0,0
2,0,0,0


In [9]:
print(env.list_containers)

[3, 2, 3, 3, 2, 2, 1, 1, 1]


### Environment testen

In [10]:
episodes = 5
for episode in range(1, episodes+1):
    state = env.reset()
    done = False
    score = 0 
    
    while not done:
        env.render()
        action = env.action_space.sample()
        n_state, reward, done, info = env.step(action)
        score+=reward
    print('Episode:{} Score:{}'.format(episode, score))
env.close()

,0,1,2
0,0,0,0
1,0,0,0
2,0,0,0


,0,1,2
0,0,0,0
1,0,0,0
2,0,0,2


,0,1,2
0,0,0,0
1,0,0,0
2,0,1,2


,0,1,2
0,0,0,2
1,0,0,0
2,0,1,2


,0,1,2
0,0,0,2
1,0,0,0
2,0,1,1


...

,0,1,2
0,3,1,0
1,0,0,0
2,2,0,2


,0,1,2
0,2,1,0
1,0,0,0
2,2,0,2


,0,1,2
0,2,1,0
1,0,0,0
2,2,3,2


,0,1,2
0,2,1,0
1,3,0,0
2,2,3,2


Episode:5 Score:3


### Environment trainen

In [11]:
#log_path = os.path.join('Training', 'Logs')
model = PPO("MlpPolicy", env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


/opt/jupyterhub/anaconda/lib/python3.9/site-packages/torch/cuda/__init__.py:83: UserWarning: CUDA initialization: CUDA driver initialization failed, you might not have a CUDA gpu. (Triggered internally at  /opt/conda/conda-bld/pytorch_1659484809662/work/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


In [12]:
model.learn(total_timesteps=100000)

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 9        |
|    ep_rew_mean     | 2.86     |
| time/              |          |
|    fps             | 1028     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 9           |
|    ep_rew_mean          | 3.22        |
| time/                   |             |
|    fps                  | 766         |
|    iterations           | 2           |
|    time_elapsed         | 5           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.015304323 |
|    clip_fraction        | 0.164       |
|    clip_range           | 0.2         |
|    entropy_loss         | -2.19       |
|    explained_variance   | -0.0627     |
|    learning_rate        | 0.

### Model gebruiken/evalueren

In [13]:
# from stable_baselines3.common.monitor import Monitor
# env.reset()
# monitor = Monitor(env)
evaluate_policy(model , env, n_eval_episodes=1, return_episode_rewards = True, render=True)

/opt/jupyterhub/anaconda/lib/python3.9/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


,0,1,2
0,0,0,0
1,0,0,0
2,0,3,0


,0,1,2
0,0,0,0
1,0,0,0
2,1,3,0


,0,1,2
0,0,0,0
1,1,0,0
2,1,3,0


,0,1,2
0,0,0,0
1,1,0,0
2,1,3,3


...

,0,1,2
0,3,0,0
1,1,0,2
2,1,3,3


,0,1,2
0,3,0,0
1,1,2,2
2,1,3,3


,0,1,2
0,3,1,0
1,1,2,2
2,1,3,3


,0,1,2
0,0,0,0
1,0,0,0
2,0,0,0


([9.0], [9])

# Conclusie
Zoals te zien is bij het testresultaat plaatst het model geen containers over elkaar, want hij kan ze alle 9 kwijt (de laatste stap wordt niet gerenderd, vandaar dat je dat niet ziet, maar de reward is 9: dus 9x +1). Dit betekent dat het simpele model binnen 100000 timesteps (mogelijk kan het sneller, daarvoor zouden we de learning curve moeten bekijken) heeft geleerd geen containers te plaatsen op een plek waar er al eentje staat.